In [2]:
import pandas as pd
import requests
from io import StringIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
# load pwlds csv files from github

files = [
    ("https://raw.githubusercontent.com/Infinitode/PWLDS/main/pwlds_very_weak.csv", 0),
    ("https://raw.githubusercontent.com/Infinitode/PWLDS/main/pwlds_weak.csv", 1),
    ("https://raw.githubusercontent.com/Infinitode/PWLDS/main/pwlds_average.csv", 2),
    ("https://raw.githubusercontent.com/Infinitode/PWLDS/main/pwlds_strong.csv", 3),
    ("https://raw.githubusercontent.com/Infinitode/PWLDS/main/pwlds_very_strong.csv", 4)
]

def load_pwlds(files, limit_per_file=None):
    X, y = [], []
    for url, label in files:
        print(f"Downloading {url.split('/')[-1]}...")
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))
        if limit_per_file:
            df = df.head(limit_per_file)
        X.extend(df['Password'].tolist())
        y.extend([label]*len(df))
    return X, y

print("Loading PWLDS passwords from GitHub...")
X, y = load_pwlds(files, limit_per_file=20000)
print(f"Dataset size: {len(X)}")

Loading PWLDS passwords from GitHub...
Dataset size: 100000


In [4]:
# train/test splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# vectorize data
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# evaluate model
y_pred = model.predict(X_test_vec)
print("\nModel Evaluation:")
print(classification_report(y_test, y_pred))


Model Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4018
           1       0.99      0.99      0.99      4015
           2       0.90      0.95      0.93      4002
           3       0.94      0.89      0.91      3987
           4       1.00      0.99      1.00      3978

    accuracy                           0.96     20000
   macro avg       0.97      0.96      0.96     20000
weighted avg       0.97      0.96      0.96     20000



In [5]:
# prediction (strength, classification, confidence)

def password_strength(password):
    classes = ['Very Weak', 'Weak', 'Average', 'Strong', 'Very Strong']
    vec = vectorizer.transform([password])
    pred = model.predict(vec)[0]
    probs = model.predict_proba(vec)[0]  
    confidence = probs[pred]*100  
    return classes[pred], confidence

In [ ]:
# interactive password strength prediction

print("-"*60)
print(" "*18 + "WEAK PASSWORD PREDICTOR")
print("-"*60)
print("This program evaluates the strength of a password and classifies it into 5 categories:Very weak, weak, average, strong, and very strong.")

while True:
    print()
    pwd = input("Enter a password (ENTER to exit): ")
    if pwd == "":
        break
    else:
        label, conf = password_strength(pwd)
        print()
        print(f"Password: \t{pwd}")
        print(f"Label: \t\t{label}")
        print(f"Confidence: \t{conf:.2f}%")

------------------------------------------------------------
                  WEAK PASSWORD PREDICTOR
------------------------------------------------------------
This program evaluates the strength of a password and classifies it into 5 categories:Very weak, weak, average, strong, and very strong.


Password: 	hi
Label: 		Very Weak
Confidence: 	100.00%


Password: 	555Ggear
Label: 		Weak
Confidence: 	93.97%


Password: 	1800hotnfun!!
Label: 		Strong
Confidence: 	98.52%


Password: 	555Ggear!!
Label: 		Weak
Confidence: 	73.80%

